## Penyematan

Dalam contoh sebelumnya, kita bekerja dengan vektor bag-of-words berdimensi tinggi dengan panjang `vocab_size`, dan secara eksplisit mengubah vektor representasi posisi berdimensi rendah menjadi representasi one-hot yang jarang. Representasi one-hot ini tidak efisien dalam penggunaan memori. Selain itu, setiap kata diperlakukan secara independen satu sama lain, sehingga vektor yang dikodekan one-hot tidak dapat mengungkapkan kesamaan semantik antar kata.

Dalam unit ini, kita akan melanjutkan eksplorasi dataset **News AG**. Untuk memulai, mari kita muat data dan mengambil beberapa definisi dari unit sebelumnya.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Apa itu embedding?

Ide **embedding** adalah merepresentasikan kata-kata menggunakan vektor padat berdimensi rendah yang mencerminkan makna semantik dari kata tersebut. Nanti kita akan membahas bagaimana membangun embedding kata yang bermakna, tetapi untuk saat ini mari kita anggap embedding sebagai cara untuk mengurangi dimensi vektor kata.

Jadi, lapisan embedding mengambil sebuah kata sebagai input, dan menghasilkan vektor output dengan ukuran `embedding_size` yang ditentukan. Dalam beberapa hal, ini sangat mirip dengan lapisan `Dense`, tetapi alih-alih mengambil vektor one-hot encoded sebagai input, lapisan ini dapat menerima nomor kata.

Dengan menggunakan lapisan embedding sebagai lapisan pertama dalam jaringan kita, kita dapat beralih dari model bag-of-words ke model **embedding bag**, di mana kita pertama-tama mengonversi setiap kata dalam teks kita ke embedding yang sesuai, lalu menghitung fungsi agregat tertentu dari semua embedding tersebut, seperti `sum`, `average`, atau `max`.

![Gambar menunjukkan pengklasifikasi embedding untuk lima kata dalam urutan.](../../../../../translated_images/id/embedding-classifier-example.b77f021a7ee67eee.webp)

Jaringan neural pengklasifikasi kita terdiri dari lapisan-lapisan berikut:

* Lapisan `TextVectorization`, yang mengambil string sebagai input, dan menghasilkan tensor nomor token. Kita akan menentukan ukuran kosakata yang masuk akal `vocab_size`, dan mengabaikan kata-kata yang jarang digunakan. Bentuk input akan menjadi 1, dan bentuk output akan menjadi $n$, karena kita akan mendapatkan $n$ token sebagai hasilnya, masing-masing berisi angka dari 0 hingga `vocab_size`.
* Lapisan `Embedding`, yang mengambil $n$ angka, dan mengurangi setiap angka menjadi vektor padat dengan panjang tertentu (100 dalam contoh kita). Dengan demikian, tensor input dengan bentuk $n$ akan diubah menjadi tensor $n\times 100$. 
* Lapisan agregasi, yang mengambil rata-rata tensor ini sepanjang sumbu pertama, yaitu akan menghitung rata-rata dari semua $n$ tensor input yang sesuai dengan kata-kata yang berbeda. Untuk mengimplementasikan lapisan ini, kita akan menggunakan lapisan `Lambda`, dan memasukkan fungsi untuk menghitung rata-rata. Output akan memiliki bentuk 100, dan ini akan menjadi representasi numerik dari seluruh urutan input.
* Pengklasifikasi linear `Dense` terakhir.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Dalam cetakan `summary`, pada kolom **output shape**, dimensi tensor pertama `None` merujuk pada ukuran minibatch, dan dimensi kedua merujuk pada panjang urutan token. Semua urutan token dalam minibatch memiliki panjang yang berbeda. Kita akan membahas cara mengatasinya di bagian berikutnya.

Sekarang, mari kita latih jaringan:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Catatan** bahwa kami sedang membangun vektorisasi berdasarkan subset data. Hal ini dilakukan untuk mempercepat proses, dan mungkin mengakibatkan situasi di mana tidak semua token dari teks kami ada dalam kosakata. Dalam kasus ini, token tersebut akan diabaikan, yang dapat menyebabkan akurasi sedikit lebih rendah. Namun, dalam kehidupan nyata, subset teks sering kali memberikan estimasi kosakata yang baik.


### Mengatasi ukuran urutan variabel

Mari kita pahami bagaimana pelatihan terjadi dalam minibatch. Dalam contoh di atas, tensor input memiliki dimensi 1, dan kita menggunakan minibatch sepanjang 128, sehingga ukuran aktual tensor adalah $128 \times 1$. Namun, jumlah token dalam setiap kalimat berbeda-beda. Jika kita menerapkan lapisan `TextVectorization` pada satu input, jumlah token yang dikembalikan akan berbeda, tergantung pada bagaimana teks tersebut di-tokenisasi:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Namun, ketika kita menerapkan vectorizer pada beberapa urutan, vectorizer harus menghasilkan tensor berbentuk persegi panjang, sehingga mengisi elemen yang tidak digunakan dengan token PAD (yang dalam kasus kita adalah nol):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Di sini kita dapat melihat penyematan:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Catatan**: Untuk meminimalkan jumlah padding, dalam beberapa kasus masuk akal untuk mengurutkan semua urutan dalam dataset berdasarkan panjang yang meningkat (atau, lebih tepatnya, jumlah token). Hal ini akan memastikan bahwa setiap minibatch berisi urutan dengan panjang yang serupa.


## Semantic embeddings: Word2Vec

Dalam contoh sebelumnya, lapisan embedding belajar memetakan kata-kata ke representasi vektor, namun, representasi tersebut tidak memiliki makna semantik. Akan lebih baik jika kita dapat mempelajari representasi vektor sehingga kata-kata yang serupa atau sinonim memiliki vektor yang dekat satu sama lain berdasarkan jarak vektor tertentu (misalnya jarak euclidean).

Untuk mencapai hal tersebut, kita perlu melatih model embedding kita terlebih dahulu pada kumpulan teks yang besar menggunakan teknik seperti [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Teknik ini didasarkan pada dua arsitektur utama yang digunakan untuk menghasilkan representasi terdistribusi dari kata-kata:

 - **Continuous bag-of-words** (CBoW), di mana kita melatih model untuk memprediksi sebuah kata berdasarkan konteks di sekitarnya. Diberikan ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, tujuan model adalah memprediksi $W_0$ dari $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** adalah kebalikan dari CBoW. Model menggunakan jendela kata-kata konteks di sekitar untuk memprediksi kata saat ini.

CBoW lebih cepat, sedangkan skip-gram lebih lambat, tetapi skip-gram lebih baik dalam merepresentasikan kata-kata yang jarang muncul.

![Gambar menunjukkan algoritma CBoW dan Skip-Gram untuk mengonversi kata-kata menjadi vektor.](../../../../../translated_images/id/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Untuk bereksperimen dengan embedding Word2Vec yang telah dilatih sebelumnya pada dataset Google News, kita dapat menggunakan pustaka **gensim**. Di bawah ini kita menemukan kata-kata yang paling mirip dengan 'neural'.

> **Note:** Saat pertama kali membuat vektor kata, proses pengunduhan dapat memakan waktu cukup lama!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Kita juga dapat mengekstrak embedding vektor dari kata, untuk digunakan dalam melatih model klasifikasi. Embedding memiliki 300 komponen, tetapi di sini kami hanya menunjukkan 20 komponen pertama dari vektor untuk kejelasan:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Hal hebat tentang embedding semantik adalah bahwa Anda dapat memanipulasi pengkodean vektor berdasarkan semantik. Sebagai contoh, kita dapat meminta untuk menemukan kata yang representasi vektornya sedekat mungkin dengan kata *raja* dan *wanita*, serta sejauh mungkin dari kata *pria*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Contoh di atas menggunakan beberapa sihir internal GenSym, tetapi logika dasarnya sebenarnya cukup sederhana. Hal yang menarik tentang embedding adalah bahwa Anda dapat melakukan operasi vektor normal pada vektor embedding, dan itu akan mencerminkan operasi pada **makna** kata. Contoh di atas dapat diekspresikan dalam bentuk operasi vektor: kita menghitung vektor yang sesuai dengan **KING-MAN+WOMAN** (operasi `+` dan `-` dilakukan pada representasi vektor dari kata-kata yang bersangkutan), dan kemudian menemukan kata terdekat dalam kamus dengan vektor tersebut:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Kami harus menambahkan koefisien kecil pada vektor *man* dan *woman* - coba hapus koefisien tersebut untuk melihat apa yang terjadi.

Untuk menemukan vektor terdekat, kami menggunakan mekanisme TensorFlow untuk menghitung vektor jarak antara vektor kami dan semua vektor dalam kosakata, lalu menemukan indeks kata dengan nilai minimum menggunakan `argmin`.


Meskipun Word2Vec tampak seperti cara yang hebat untuk mengekspresikan semantik kata, metode ini memiliki banyak kekurangan, termasuk yang berikut:

* Baik model CBoW maupun skip-gram adalah **predictive embeddings**, dan hanya mempertimbangkan konteks lokal. Word2Vec tidak memanfaatkan konteks global.
* Word2Vec tidak mempertimbangkan **morfologi** kata, yaitu fakta bahwa makna kata dapat bergantung pada bagian-bagian tertentu dari kata, seperti akar kata.

**FastText** mencoba mengatasi keterbatasan kedua ini, dan mengembangkan Word2Vec dengan mempelajari representasi vektor untuk setiap kata serta n-gram karakter yang ditemukan dalam setiap kata. Nilai dari representasi ini kemudian dirata-rata menjadi satu vektor pada setiap langkah pelatihan. Meskipun ini menambah banyak perhitungan tambahan selama pretraining, metode ini memungkinkan embeddings kata untuk menyandikan informasi sub-kata.

Metode lain, **GloVe**, menggunakan pendekatan yang berbeda untuk embeddings kata, berdasarkan faktorisasi matriks kata-konteks. Pertama, metode ini membangun matriks besar yang menghitung jumlah kemunculan kata dalam berbagai konteks, lalu mencoba merepresentasikan matriks ini dalam dimensi yang lebih rendah dengan cara yang meminimalkan kehilangan rekonstruksi.

Perpustakaan gensim mendukung embeddings kata tersebut, dan Anda dapat bereksperimen dengan mengubah kode pemuatan model di atas.


## Menggunakan embedding yang sudah dilatih sebelumnya di Keras

Kita dapat memodifikasi contoh di atas untuk mengisi matriks dalam lapisan embedding kita dengan embedding semantik, seperti Word2Vec. Kosakata dari embedding yang sudah dilatih sebelumnya dan korpus teks kemungkinan besar tidak akan cocok, jadi kita perlu memilih salah satu. Di sini kita akan mengeksplorasi dua opsi yang mungkin: menggunakan kosakata tokenizer, dan menggunakan kosakata dari embedding Word2Vec.

### Menggunakan kosakata tokenizer

Saat menggunakan kosakata tokenizer, beberapa kata dari kosakata akan memiliki embedding Word2Vec yang sesuai, dan beberapa lainnya akan hilang. Mengingat bahwa ukuran kosakata kita adalah `vocab_size`, dan panjang vektor embedding Word2Vec adalah `embed_size`, lapisan embedding akan direpresentasikan oleh matriks bobot dengan bentuk `vocab_size`$\times$`embed_size`. Kita akan mengisi matriks ini dengan menelusuri kosakata:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Untuk kata-kata yang tidak ada dalam kosakata Word2Vec, kita dapat membiarkannya sebagai nol, atau menghasilkan vektor acak.

Sekarang kita dapat mendefinisikan lapisan embedding dengan bobot yang telah dilatih sebelumnya:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Catatan**: Perhatikan bahwa kami menetapkan `trainable=False` saat membuat `Embedding`, yang berarti kami tidak melatih ulang lapisan Embedding. Hal ini mungkin menyebabkan akurasi sedikit lebih rendah, tetapi mempercepat proses pelatihan.

### Menggunakan kosakata embedding

Salah satu masalah dengan pendekatan sebelumnya adalah kosakata yang digunakan dalam TextVectorization dan Embedding berbeda. Untuk mengatasi masalah ini, kita dapat menggunakan salah satu solusi berikut:
* Melatih ulang model Word2Vec pada kosakata kita.
* Memuat dataset kita dengan kosakata dari model Word2Vec yang telah dilatih sebelumnya. Kosakata yang digunakan untuk memuat dataset dapat ditentukan selama proses pemuatan.

Pendekatan kedua tampaknya lebih mudah, jadi mari kita implementasikan. Pertama-tama, kita akan membuat lapisan `TextVectorization` dengan kosakata yang telah ditentukan, diambil dari embedding Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Perpustakaan word embeddings gensim memiliki fungsi yang praktis, `get_keras_embeddings`, yang secara otomatis akan membuat lapisan embeddings Keras yang sesuai untuk Anda.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Salah satu alasan mengapa kita tidak melihat akurasi yang lebih tinggi adalah karena beberapa kata dari dataset kita tidak ada dalam kosakata GloVe yang telah dilatih sebelumnya, sehingga kata-kata tersebut pada dasarnya diabaikan. Untuk mengatasi hal ini, kita dapat melatih embedding kita sendiri berdasarkan dataset kita.


## Embedding Kontekstual

Salah satu keterbatasan utama dari representasi embedding yang dilatih sebelumnya seperti Word2Vec adalah meskipun mereka dapat menangkap sebagian makna dari sebuah kata, mereka tidak dapat membedakan antara makna yang berbeda. Hal ini dapat menyebabkan masalah pada model lanjutan.

Sebagai contoh, kata 'play' memiliki makna yang berbeda dalam dua kalimat berikut:
- Saya pergi ke sebuah **play** di teater.
- John ingin **play** dengan teman-temannya.

Embedding yang dilatih sebelumnya yang kita bahas merepresentasikan kedua makna kata 'play' dalam embedding yang sama. Untuk mengatasi keterbatasan ini, kita perlu membangun embedding berdasarkan **model bahasa**, yang dilatih pada korpus teks yang besar, dan *memahami* bagaimana kata-kata dapat disusun dalam berbagai konteks. Membahas embedding kontekstual berada di luar cakupan tutorial ini, tetapi kita akan kembali membahasnya saat berbicara tentang model bahasa di unit berikutnya.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
